In [1]:
client = QdrantClient(path="/tmp/langchain_qdrant")
# Realiza operaciones con el cliente

# Cerrar la sesión
client.close()

NameError: name 'QdrantClient' is not defined

# Lesson 4: Retrieval Methods and Vector Databases

**Objective**: Build a retrieval system that efficiently searches for relevant document chunks.

**Topics**:
- Sparse vs. dense retrieval methods
- Hybrid search methods (e.g., combining BM25 with dense retrieval)
- Overview of vector databases: Milvus, Faiss, Qdrant

**Practical Task**: Set up a vector database and implement a retrieval method.

**Resources**:
- What is a vector database
- Choosing a vector database


#### Load the dataset

In [3]:
from langchain_community.document_loaders import PyPDFLoader
from dotenv import load_dotenv

load_dotenv()

file_path = (
    "../Castro_Cofre_Zurita/data/Regulaciones cacao y chocolate 2003.pdf"
)
loader = PyPDFLoader(file_path)
docs = loader.load_and_split()

### Embeddings function

In [4]:
from langchain_huggingface import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
embedded_document = embedding_model.embed_query(docs[0].page_content)
embedded_document[:3]

c:\Users\nicol\OneDrive\MIA\Cursos\SEGUNDO BIMESTRE\PROYECTO APLICADO\PROYECTO\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[0.02872576005756855, -0.047862425446510315, 0.005437064450234175]

# A first approach

In [2]:
client = QdrantClient()
# Realiza operaciones con el cliente

# Cerrar la sesión
client.close()

NameError: name 'QdrantClient' is not defined

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

c:\Users\nicol\OneDrive\MIA\Cursos\SEGUNDO BIMESTRE\PROYECTO APLICADO\PROYECTO\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
client = QdrantClient(path="/tmp/langchain_qdrant")

In [5]:
client.delete_collection(collection_name="demo_collection")

True

In [10]:
client.create_collection(
    collection_name="demo_collection1",
    vectors_config=VectorParams(size=768, distance=Distance.COSINE),
)

vector_store = QdrantVectorStore(
    client=client,
    collection_name="demo_collection1",
    embedding=embedding_model,
)

In [11]:
vector_store.add_documents(docs)


['f276333f2ffc44aeb426ebbd7ad90e6a',
 'b25ef1afcdf04d6bb53d4db52eda5e25',
 'bff9f875473f4a5987e75cd6124a984b',
 '0be67082578840648ffa303040d2a30b',
 '95911e0a0ce44cbaabdcd5369ae364c0',
 'b1f904847d08408e87a5007f8359f80e',
 'de8906d5ca034260bd3e2151b687f4fe',
 '3d0f854107524ac5aa76042e82cd5b11',
 'ffc5500dbbc344feb34a516cd6e4e506',
 '5dfedd67ba854cabb815b2e3c4e24fb3',
 '9974f3ef9d2547469b79b8b95edd9551',
 'd89cb7a53c924d028d45b089ca181c08',
 '5e53e2cdabda485a903e86a2f801ced0']

In [ ]:
client.scroll(collection_name="demo_collection", limit=3)

# Dense search

In [12]:
from langchain_qdrant import RetrievalMode

qdrant = QdrantVectorStore.from_documents(
    docs,
    embedding=embedding_model,
    location=":memory:",
    collection_name="my_documents",
    retrieval_mode=RetrievalMode.DENSE,
)

query = "What did the president say about Ketanji Brown Jackson"
found_docs = qdrant.similarity_search(query)

In [ ]:
found_docs

# Sparse Vector Search

To search with only sparse vectors,

The retrieval_mode parameter should be set to RetrievalMode.SPARSE.
An implementation of the SparseEmbeddings interface using any sparse embeddings provider has to be provided as value to the sparse_embedding parameter.
The langchain-qdrant package provides a FastEmbed based implementation out of the box.

In [13]:
from langchain_qdrant import FastEmbedSparse, RetrievalMode

sparse_embeddings = FastEmbedSparse(model_name="Qdrant/bm25", cache_dir="cache")

qdrant = QdrantVectorStore.from_documents(
    docs,
    embedding=embedding_model,
    sparse_embedding=sparse_embeddings,
    location=":memory:",
    collection_name="my_documents",
    retrieval_mode=RetrievalMode.SPARSE,
)

query = "What is chocolate?"
found_docs = qdrant.similarity_search(query)

Fetching 29 files:   0%|          | 0/29 [00:00<?, ?it/s]c:\Users\nicol\OneDrive\MIA\Cursos\SEGUNDO BIMESTRE\PROYECTO APLICADO\PROYECTO\.venv\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\nicol\OneDrive\MIA\Cursos\SEGUNDO BIMESTRE\PROYECTO APLICADO\PROYECTO\Castro_Cofre_Zurita\cache\models--Qdrant--bm25. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-y

In [ ]:
found_docs

# Hybrid Search

To perform a hybrid search using dense and sparse vectors with score fusion,

The retrieval_mode parameter should be set to RetrievalMode.HYBRID.
A dense embeddings value should be provided to the embedding parameter.
An implementation of the SparseEmbeddings interface using any sparse embeddings provider has to be provided as value to the sparse_embedding parameter.
Note that if you've added documents with the HYBRID mode, you can switch to any retrieval mode when searching. Since both the dense and sparse vectors are available in the collection.

In [14]:
from langchain_qdrant import FastEmbedSparse, RetrievalMode

sparse_embeddings = FastEmbedSparse(model_name="Qdrant/bm25")

qdrant = QdrantVectorStore.from_documents(
    docs,
    embedding=embedding_model,
    sparse_embedding=sparse_embeddings,
    location=":memory:",
    collection_name="my_documents",
    retrieval_mode=RetrievalMode.HYBRID,
)

query = "What did the president say about Ketanji Brown Jackson"
found_docs = qdrant.similarity_search(query)

Fetching 29 files:   0%|          | 0/29 [00:00<?, ?it/s]c:\Users\nicol\OneDrive\MIA\Cursos\SEGUNDO BIMESTRE\PROYECTO APLICADO\PROYECTO\.venv\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\nicol\AppData\Local\Temp\fastembed_cache\models--Qdrant--bm25. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Fet

In [ ]:
found_docs

In [ ]:
#If you want to execute a similarity search and receive the corresponding scores you can run:
results = vector_store.similarity_search_with_score(
    query="What is chocolate?", k=1
)
for doc, score in results:
    print(f"* [SIM={score:3f}] {doc.page_content} [{doc.metadata}]")

# Metadata filtering

In [ ]:
from qdrant_client.http import models

results = vector_store.similarity_search(
    query="What is chocolate?",
    k=1,
    filter=models.Filter(
        should=[
            models.FieldCondition(
                key="page",
                match=models.MatchValue(
                    value="5"
                ),
            ),
        ]
    ),
)
for doc in results:
    print(f"* {doc.page_content} [{doc.metadata}]")

In [ ]:
results

## Query by turning into a retriever

In [ ]:
retriever = vector_store.as_retriever(search_type="mmr", search_kwargs={"k": 5})
retriever.invoke("What is chocolate?")
